In [7]:
# Import required libraries

import urllib.request, urllib.parse, urllib.error
import bs4 as bs
import ssl
import numpy as np 
import pandas as pd 


Use the Notebook to build the code to scrape the following Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

In [8]:

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


html = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', context=ctx).read()
soup = bs.BeautifulSoup(html, 'html.parser')

# Retrieve all of the table rows and put them into a pandas dataframe
row_list = []
table_rows = soup('tr')
for row in table_rows:
    td = row.find_all('td')
    row = [tr.text for tr in td]
    row_list.append(row)
Toronto_DF = pd.DataFrame(row_list)



Process the scraped dataframe to satisfy the following requirements:

1- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.  
2- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  
3- More than one neighborhood can exist in one postal code area.   
4- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  
5- Use the .shape method to print the number of rows of your dataframe.

In [9]:
#Clean up the dataframe

#Just keep required columns
Toronto_DF = Toronto_DF[[0,1,2]]

#Rename columns
Toronto_DF.columns=["PostalCode", "Borough", "Neighborhood"]

#Remove any '\n' in the strings
Toronto_DF = Toronto_DF.replace('\n','', regex=True)

#Drop rows that are not required
Toronto_DF = Toronto_DF.dropna()
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'Not assigned']
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'B' ]
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'NL']
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'NS']

#Group rows if required
Toronto_DF['Neighborhood'] = Toronto_DF.groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
Toronto_DF = Toronto_DF.drop_duplicates()

Toronto_DF.shape

(103, 3)

Get the latitude and the longitude coordinates of each postal code, using geocoder:

In [10]:
#Install geocoder (uncomment the following line to install geocoder if is not installed aleady)
#!conda install -c conda-forge geocoder



In [11]:
# import geocoder
import geocoder 

# Define a function to convert PostalCode to lat and Long
def Postal_Code_to_LatLong(postal_code):
    
 # initialize your variable to None
 lat_lng_coords = None
    
    
# Loop until you get the coordinates
 while(lat_lng_coords is None):
     g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
     lat_lng_coords = g.latlng
 return lat_lng_coords


In [12]:
# Use the defined function to add latitude and longitude for each postal code
Toronto_DF[['Latitude', 'Longitude']] =  pd.DataFrame(Toronto_DF['PostalCode'].apply(Postal_Code_to_LatLong).to_list(), index=Toronto_DF.index)

Toronto_DF

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.752935,-79.335641
4,M4A,North York,Victoria Village,43.728102,-79.311890
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
9,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953
10,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
12,M3B,North York,Don Mills,43.748900,-79.357220
13,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
